# 감정 분석

<br>

- 파이프라인이 처리하는 어떤것이든 각 토큰에는 어떠한 정보가 담겨있음.
- 단어의 감정(sentiment), 그 단어가 불러 일으키는 전반적인 느낌은 그런 정보의 중요한 일부분.
- 감정분석은 NLP의 일반적인 응용 중 하나.
    - 이를 응용해 스팸 필터링, 챗봇응용 등이 있음.

<br>

- 파이프라인이 텍스트 조각에 담긴 감정을 분석해 수치화하려면 어떻게 해야 할까. 크게 2가지 접근방식.
    1. 사람이 직접 작성한 **규칙 기반 알고리즘** 사용 $\rightarrow$ **발견법(heuristics)에 기초한 감성 분석**
        - 텍스트에서 **특정 키워드**를 찾고, **키워드 들에 부여된 수치 점수를 취합**.
        - **특정 단어와 점수 쌍을 담은 Dict 자료구조**가 필요. 사람이 직접 만들어야 함.
        - 주로 **VADER**를 이용.
    2. 컴퓨터가 자료로부터 직접 배우는 **ML모형** 사용
        - **분류명이 붙은** 문장/문서집합 을 통해 ML모형 훈련, 규칙 생성.
        - ML모형은 입력 텍스트에 대한 수치적 감정 점수를 출력하도록 훈련.
        - **'정답'에 해당하는 감정 점수가 부여된 텍스트 조각들로 이뤄진 대량의 자료가 필요**한 지도학습 모형.
            - 주로 해시태그가 많이 붙는 tweet 데이터가 많이 쓰임.
            - 또는 의견과 함께 별점 평가를 받아 활용할 수도 있음.
        - **Naive-Bayes**라 불리는 토큰 기반 ML 알고리즘을 주로 이용.

## VADER : 규칙기반 감정분석기

<br>

- **V**alence **A**ware **D**ictionary for s**E**ntiment **R**easoning (감정 추론을 위한 결합가 인식 사전)의 약자.
- NLTK의 경우 nltk.sentiment.vader 를 통해 사용 가능.
- vaderSentiment 라는 패키지도 있음. 예제는 이걸 쓸것.

        pip install vaderSentiment

In [1]:
!pip install vaderSentiment

You should consider upgrading via the 'c:\users\skdbs\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sa = SentimentIntensityAnalyzer()
sa.lexicon # lexicon에 토큰-감정 점수 쌍들이 들어있음.

# VADER는 이모티콘 사용. 분석이 잘 되려면 토큰 생성기가 문장 부호들을 제거하지 않는것이 바람직.
sa.polarity_scores(text={
    ':(' : -1.9,
    ':)' : 2.0,
    'pls' : 0.3,
    'plz' : 0.3,
    'great' : 3.1
})
[(tok, score) for tok, score in sa.lexicon.items() if " " in tok]

[("( '}{' )", 1.6),
 ("can't stand", -2.0),
 ('fed up', -1.8),
 ('screwed up', -1.5)]

In [2]:
sa.polarity_scores(text="Python is very readable and it's great for NLP.")

{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'compound': 0.6249}

    VADER 알고리즘음 감정의 세기(intensity)를 긍정(pos), 부정(neg), 중립(neu)로 분류.
    이 3개의 점수를 담은 하나의 복합 자료 구조를 return 함.

In [3]:
sa.polarity_scores(text="Python is not a bad choice for most applications.")

{'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.431}

    not 같은 부정어를 상당히 잘 처리함.
    not bad의 긍정 점수는 great보다 약간만 낮을 정도로 높은 편.
    VADER의 내장 토큰 생성기는 자신의 어휘집엔 없는 단어들을 모두 무시, n-gram은 전혀 고려 X.

In [4]:
corpus = ["Absolutely perfect!! Love it! :-) :-) :-)",
          "Horrible! Completely useless :(",
          "It was OK. Some good and some bad things."]
for doc in corpus:
    scores = sa.polarity_scores(doc)
    print('{:+}: {}'.format(scores['compound'], doc))

+0.9455: Absolutely perfect!! Love it! :-) :-) :-)
-0.8768: Horrible! Completely useless :(
-0.1531: It was OK. Some good and some bad things.


    상당히 잘 동작하고 있음.

- VADER의 유일한 단점은 **모든 단어가 아니라 약 7500개의 단어만을 고려**한다는 것.
- 모든 단어로 감정 점수를 측정해야 한다면 할 일이 엄청나게 늘어남.
    - 일일이 점수를 매기거나, lexicon 사전 자료 구조에 수많은 커스텀 단어를 추가해야 함.
- ***규칙 기반 접근방식은 대상 자연어를 이해하지 못하면 거의 불가능***한 일.
- 그래서 **기계학습에 기초한 감정 분석기를 사용**하는 것이 편리하다.

## Naive-Bayes Model

<br>

- 주어진 문서 집합에서 목표 변수를 예측하는 키워드들을 찾음.
- 감정분석의 경우, 모형의 **목표 변수는 평가하고자 하는 감정**.
    - 즉, 해당 **감정을 예측하는 단어들을 분류**.
- 미리 만들어진 사전에 의존하지 않고, 임의의 문제에 대해 최선의 감정 점수를 찾아냄.
- 마찬가지로 Train Set이 필요. 각 내용에 대한 분류명이 붙은 대량의 텍스트 문서가 필요.
    - NLPIA에 이런 자료 집합이 있다.

In [ ]:
!pip install nlpia # 데스크탑은 이미 있음.

In [5]:
from nlpia.data.loaders import get_data

movies = get_data('hutto_movies')
movies.head().round(2), movies.describe().round(2)

C:\Users\skdbs\Anaconda3\lib\site-packages\pugnlp\constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
C:\Users\skdbs\Anaconda3\lib\site-packages\pugnlp\constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
C:\Users\skdbs\Anaconda3\lib\site-packages\pugnlp\tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
C:\Users\skdbs\Anaconda3\lib\site-packages\pugnlp\util.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
INFO:nlpia.constants:Startin

(    sentiment                                               text
 id                                                              
 1        2.27  The Rock is destined to be the 21st Century's ...
 2        3.53  The gorgeously elaborate continuation of ''The...
 3       -0.60                     Effective but too tepid biopic
 4        1.47  If you sometimes like to go to the movies to h...
 5        1.73  Emerges as something rare, an issue movie that...,
        sentiment
 count   10605.00
 mean        0.00
 std         1.92
 min        -3.88
 25%        -1.77
 50%        -0.08
 75%         1.83
 max         3.94)

    describe로 본 결과, 영화 평점은 -4 ~ +4 사이인 것으로 보임.
  
  
<br>

- 이 영화평 텍스트들을 토큰화 해 단어 모음을 생성해보자. 

In [6]:
import pandas as pd
pd.set_option('display.width', 75) # DF내용을 좀더 보기좋게 row 너비 늘림.

In [7]:
from nltk.tokenize import casual_tokenize # 이모지, 비표준적인 부호, 비속어를 좀더 잘 처리함.
from collections import Counter

bags_of_words = []
for text in movies.text:
    # text를 토큰화 한 내용들의 수를 세고, Dict로 담아 전달
    bags_of_words.append(Counter(casual_tokenize(text)))

# Dict를 받아 모든 key-value로 하나의 테이블을 생성.
# Key는 Column이 되고, key가 없다면 해당 테이블 칸은 NaN이 됨.
# 그러므로 이런 결측값들을 0으로 채우는 것이 좋을 것.
df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)
print(df_bows.shape)
df_bows.head()

(10605, 20756)


,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Ill,slummer,Rashomon,dipsticks,Bearable,Staggeringly,’,ve,muttering,dissing
0,1,1,1,1,2,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


    생각한 것 보다 큼.
    왜? 불용어 처리도 안했고, 어간/표제어 추출도 안했으니까.

In [8]:
df_bows.head()[list(bags_of_words[0].keys())]

,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Schwarzenegger,",",Jean,Claud,Van,Damme,or,Steven,Segal,.
0,1,1,1,1,2,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


    아주 간단하게 나이브-베이즈 모형을 통해 감정 분석을 해보자.

In [14]:
type(movies)

pandas.core.frame.DataFrame

In [10]:
from sklearn.naive_bayes import MultinomialNB # 결과는 여러개이니 다항분포 모형 사용.

nb = MultinomialNB()
# train
nb = nb.fit(df_bows, movies.sentiment > 0)
movies['predicted_sentiment'] = nb.predict(df_bows) * 8 - 4
movies['error'] = (movies.predicted_sentiment - movies.sentiment).abs() # 오차값 계산
# movies.error.mean().round(1)
movies['sentiment_ispositive'] = (movies.sentiment > 0).astype(int)
movies['predicted_ispositive'] = (movies.predicted_sentiment > 0).astype(int)
movies["""sentiment predicted_sentiment sentiment_ispositive predicted_ispositive""".split()].head(8)

,sentiment,predicted_sentiment,sentiment_ispositive,predicted_ispositive
id,,,,
1,2.266667,4,1,1
2,3.533333,4,1,1
3,-0.600000,-4,0,0
4,1.466667,4,1,1
5,1.733333,4,1,1
6,2.533333,4,1,1
7,2.466667,4,1,1
8,1.266667,-4,1,0


    왜 컬럼추가가 안되는걸까
    predict_proba 말고 predict...

In [11]:
(movies.predicted_ispositive == movies.sentiment_ispositive).sum() / len(movies)

0.9344648750589345

    긍정적 추천 평가는 93%의 경우 정확함을 알 수 있음.

- VADER 접근방식이라면 몇천개의 단어를 고르고, 각각에 감정 점수를 매겨야 했을 것.
- 분류명이 붙은 텍스트만을 이용해 ML모델을 굴리니 괜찮게 분류했다.

<br>

- 영화 평점이 아닌 상품평 같은 완전히 다른 종류의 감정 점수를 예측하게 되면 어떻게 될까?
- 앞선 영화 분류는 추천/비추천 중 하나로 분류하도록 훈련했음.
- 상품평에 적용하면 어떤 결과가 나올까?

    잔에러 너무많이나서 시간 너무버림. 

- 나이브 베이즈 모형은 VADER보다 부정어를 잘 다루지 못함.
    - **부정어와 그것이 수식하는 단어를 n-gram으로 묶어서 토큰화** 해야 함.